In [93]:
#word2vec
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.test.utils import datapath
from collections import Counter
import operator
import collections


#import fasttext.util

#utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

#src
#root_project = "/content/ReSt/"
root_project = "/Users/Alessandro/Dev/repos/ReSt/"
#root_project = "/home/jupyter/ReSt/"
sys.path.append(root_project)
from src.data.utils import load_csv_to_dict, dtype, dtype_transformation, set_unkmark_token
from src.data.word_embedding import get_index_key_association, get_int_seq, build_keras_embedding_matrix, get_data_to_emb

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Dataset

In [63]:
#path
dataset_path    = root_project + 'dataset/haspeede2/preprocessed/dev/dev.csv'
w2v_bin_path    = root_project + 'results/model/word2vec/twitter128.bin'
#w2v_bin_path    = root_project + 'results/model/word2vec/tweets_2019_Word2Vect.bin'

In [79]:
dataset = load_csv_to_dict(dataset_path)
senteces = dataset["tokens"]

Converting data 'lemma' String in list of String
Converting data 'pos' String in list of String
Converting data 'dep' String in list of String
Converting data 'word_polarity' String in list of String
Converting data 'tokens' String in list of String
Converting data 'stem' String in list of String


In [24]:
dtype(dataset)

id                   <class 'int'>
text                 <class 'str'>
hs                   <class 'int'>
stereotype           <class 'int'>
rawtext              <class 'str'>
text_length          <class 'int'>
hashtags             <class 'int'>
%CAPS-LOCK words     <class 'int'>
esclamations         <class 'int'>
questions            <class 'int'>
lemma                <class 'list'>
pos                  <class 'list'>
dep                  <class 'list'>
word_polarity        <class 'list'>
tokens               <class 'list'>
stem                 <class 'list'>
%bad_words           <class 'int'>


In [26]:
sentece_i = 53
print("Examples sentence: {}".format(dataset["rawtext"][sentece_i]))
print("To tokens: {}".format(dataset["tokens"][sentece_i]))

Examples sentence: +++#Siria🇸🇾 Evacuati civili dalla città terroristi #IS avanzano dopo violenti scontri con Esercito #siriano a #Palmira ma non è finita+++ 
To tokens: ['+', '+', '+', '<', 'Siria', '>', 'bandiera', 'siria', 'Evacuati', 'civili', 'dalla', 'città', 'terroristi', '<', 'Is', '>', 'avanzano', 'dopo', 'violenti', 'scontri', 'con', 'Esercito', '<', 'siriano', '>', 'a', '<', 'Palmira', '>', 'ma', 'non', 'è', 'finita', '+', '+', '+']


## useful information

In [80]:
#data
n_sentences = len(senteces)
unique_words = set([word for words in senteces for word in words])
unique_words_freq = dict(Counter(i for sub in senteces for i in set(sub)))
n_unique_words = len(unique_words)

#print data
print(" - #sentences: {}".format(n_sentences))
print(" - Unique word on the datset: {}".format(n_unique_words))

 - #sentences: 6839
 - Unique word on the datset: 20914


## W2V

In [64]:
token_setences = dataset["tokens"]

In [71]:
#w2v_model = Word2Vec.load(w2v_bin_path)
wv = KeyedVectors.load_word2vec_format(datapath(w2v_bin_path), binary=True)

In [72]:
#len(w2v_model.wv.vocab.keys())
len(wv.vocab.keys())

1170776

In [83]:
know_words   = []
unknow_words = []
for word in unique_words:
    if word in wv.vocab.keys():
        know_words.append(word)
    else:
        unknow_words.append(word)
print("know words: {}".format(len(know_words)))
print("unknow words: {}".format(len(unknow_words)))

know words: 19221
unknow words: 1693


In [97]:
unknow_words_freq = {word: unique_words_freq[word] for word in unknow_words}                     
unknow_words_freq_sorted = sorted(unknow_words_freq.items(),key=operator.itemgetter(1),reverse=True)
unknow_words_freq_sorted

[('@dg', 1252),
 ('…', 421),
 ('“', 266),
 ('”', 243),
 ("e'", 76),
 ('–', 65),
 ('Mcinque', 42),
 ('farsi', 31),
 ("po'", 29),
 (":'", 25),
 ('farli', 14),
 ("perche'", 13),
 ("piu'", 13),
 ('integrarsi', 12),
 ('farlo', 11),
 ("Perche'", 11),
 ("puo'", 10),
 ('mantenerli', 9),
 ('mcinque', 9),
 ('farla', 9),
 ('webitalia', 7),
 ("Piu'", 7),
 ('chiamarli', 7),
 ('prendersi', 7),
 (",'", 7),
 ('mandarli', 6),
 ('aiutarli', 6),
 ('`', 6),
 (">'", 6),
 ('portarli', 6),
 ('dargli', 5),
 ('__LONG', 5),
 ("sta'", 5),
 ('lasciarli', 5),
 ('difendersi', 5),
 ('LONG__', 5),
 ("migranti'", 5),
 ("di'", 5),
 ('dirlo', 5),
 ('opporsi', 5),
 ('prostituirsi', 5),
 ('•', 5),
 ('rimandarli', 5),
 ('sottomettersi', 4),
 ('centrisociale', 4),
 ('derubò', 4),
 ("cio'", 4),
 ('rubargli', 4),
 ('difenderci', 4),
 ('metterci', 4),
 ('darlo', 4),
 ('darci', 4),
 ("citta'", 4),
 ("gia'", 4),
 ("da'", 4),
 ('accoglierne', 4),
 (':@', 4),
 ('salvarci', 4),
 ('prenderli', 4),
 ('farvi', 3),
 ('usarli', 3),
 ('s